<a href="https://colab.research.google.com/github/adityasharma10699/demopygit/blob/main/LimitPrediction_LR%2BRFR%2BSVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
import plotly.graph_objs as go
import plotly.figure_factory as ff

# Importing dataset and examining it
dataset = pd.read_csv("/content/drive/MyDrive/AS & ML/Regression/LimitPrediction.csv")
pd.set_option('display.max_columns', None) # to make sure you can see all the columns in output window
print(dataset.head())
print(dataset.shape)
print(dataset.info())
print(dataset.describe())

# Converting Categorical features into Numerical features
dataset['Gender'] = dataset['Gender'].map({' Male': 1, 'Female':0})
dataset['Student'] = dataset['Student'].map({'Yes': 1, 'No':0})
dataset['Married'] = dataset['Married'].map({'Yes': 1, 'No':0})
print(dataset.info())

categorical_features = ['Ethnicity']
final_data = pd.get_dummies(dataset, columns = categorical_features, drop_first = True)
print(final_data.info())
print(final_data.head(2))

# Plotting Correlation Heatmap
corrs = dataset.corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)
figure.show()

# Dividing dataset into label and feature sets
X = final_data.drop(['Limit','Customer Id'], axis = 1) # Features
Y = final_data['Limit'] # Labels
print(type(X))
print(type(Y))
print(X.shape)
print(Y.shape)

# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

# Linear Regression with Regularization
# Tuning the SGDRegressor parameters 'eta0' (learning rate) and 'max_iter', along with the regularization parameter alpha using Grid Search
sgdr = SGDRegressor(random_state = 1, penalty = 'elasticnet')
grid_param = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100], 'l1_ratio': [0,0.25,0.5,0.75,1]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(320-1)/(320-11-1)
print("Adjusted r2: ", Adj_r2)

'''
Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

where, n = number of observations in training data, p = number of features
'''

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

#################################################################################
# Implementing Random Forest Regression
# Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
rfr = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
grid_param = {'n_estimators': [10,20,30,40,50,100]}

gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(320-1)/(320-11-1)
print("Adjusted r2: ", Adj_r2)

'''
Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

where, n = number of observations in training data, p = number of features
'''

featimp = pd.Series(gd_sr.best_estimator_.feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importances list for the best model
print(featimp)

# # Selecting features with higher sifnificance and redefining feature set
# X_ = dataset[['GrLivArea', 'YearBuilt', 'TotalBsmtSF', 'GarageArea']]

# feature_scaler = StandardScaler()
# X_scaled_ = feature_scaler.fit_transform(X_)

# # Tuning the random forest parameter 'n_estimators' and implementing cross-validation using Grid Search
# rfr = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
# grid_param = {'n_estimators': [50,100,150,200,250]}

# gd_sr = GridSearchCV(estimator=rfr, param_grid=grid_param, scoring='r2', cv=5)

# gd_sr.fit(X_scaled_, Y)

# best_parameters = gd_sr.best_params_
# print("Best parameters: ", best_parameters)

# best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
# print("r2: ", best_result)

# Adj_r2 = 1-(1-best_result)*(320-1)/(320-11-1)
# print("Adjusted r2: ", Adj_r2)

# '''
# Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

# where, n = number of observations in training data, p = number of features
# '''
####################################################################################
# Implementing Support Vector Regression
# Tuning the SVR parameters 'kernel', 'C', 'epsilon' and implementing cross-validation using Grid Search
svr = SVR()
grid_param = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [.001,.01, 0.1, 1, 10,100]}

gd_sr = GridSearchCV(estimator=svr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, Y)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("r2: ", best_result)

Adj_r2 = 1-(1-best_result)*(320-1)/(320-11-1)
print("Adjusted r2: ", Adj_r2)

   Customer Id   Income  Limit  Rating  Cards  Age  Education  Gender Student  \
0            1   14.891   3606     283      2   34         11    Male      No   
1            2  106.025   6645     483      3   82         15  Female     Yes   
2            3  104.593   7075     514      4   71         11    Male      No   
3            4  148.924   9504     681      3   36         11  Female      No   
4            5   55.882   4897     357      2   68         16    Male      No   

  Married  Ethnicity  Balance  
0     Yes  Caucasian      333  
1     Yes      Asian      903  
2      No      Asian      580  
3      No      Asian      964  
4     Yes  Caucasian      331  
(400, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Customer Id  400 non-null    int64  
 1   Income       400 non-null    float64
 2   Limit        400 non-null    int6

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
(400, 11)
(400,)


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of ite

Best parameters:  {'alpha': 0.01, 'eta0': 0.001, 'l1_ratio': 1, 'max_iter': 40000}
r2:  0.99558670155358
Adjusted r2:  0.9954290837519221
Intercept:  [4735.59916222]
               Features  Coefficients
1                Rating   2015.496386
8               Balance    211.990088
0                Income    132.940775
4             Education     12.300756
9       Ethnicity_Asian      6.890135
3                   Age      2.504680
10  Ethnicity_Caucasian     -1.038098
5                Gender     -2.351009
7               Married    -13.372841
6               Student    -72.400242
2                 Cards   -101.101276
Best parameters:  {'n_estimators': 100}
r2:  0.976078072899379
Adjusted r2:  0.975223718360071
Rating                 0.489218
Balance                0.243054
Income                 0.200662
Age                    0.028237
Education              0.012596
Cards                  0.011347
Student                0.004760
Ethnicity_Caucasian    0.002867
Gender                 0.00

In [ ]:
from google.colab import drive
drive.mount('/content/drive')